# 必要なパッケージのインストール

In [1]:
#!pip install pandas
#!pip install datapackage
#!pip install SQLAlchemy
#!pip install tableschema-sql

# 銘柄データを取得してSQLLiteで保存
# 参考（https://rainbow-engine.com/python-sqlalchemy-importcsv/）

In [119]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///periodic-table-datapackage_company.db')

In [120]:
from datapackage import Package
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')
#package.resource_names

In [121]:
package.save(storage='sql', engine=engine)

Storage <Engine(sqlite:///periodic-table-datapackage_company.db)/None>

# 銘柄情報テーブルの内容確認

In [123]:
from sqlalchemy import Table, Column, Integer, String, MetaData
tbl = Table('constituents_csv', MetaData(), autoload_with=engine)

In [124]:
print(tbl.c.keys())

['Symbol', 'Name', 'Sector']


In [125]:
print(list(engine.execute('SELECT count() from constituents_csv')))

[(505,)]


# SQLで銘柄情報を取得

In [127]:
#print(list(engine.execute('SELECT * from constituents_csv where Name like \'%Bank%\'')))
print(list(engine.execute('SELECT Symbol from constituents_csv limit 10')))

[('MMM',), ('AOS',), ('ABT',), ('ABBV',), ('ABMD',), ('ACN',), ('ATVI',), ('ADBE',), ('AAP',), ('AMD',)]


# DataFrame形式で作成

In [214]:
import pandas as pd
df = pd.DataFrame(list(engine.execute('SELECT * from constituents_csv')))
df.columns = ['Symbol', 'Name', 'Sector']
df.set_index("Symbol",inplace=True)

In [216]:
df.to_csv("./output/s_and_p_stock_list.csv")

In [33]:
# 追加情報付きのデータ取得

In [2]:
from datapackage import Package
package = Package('https://datahub.io/core/s-and-p-500-companies-financials/datapackage.json')

In [34]:
## 取得したデータ内容の確認とCS

In [17]:
package.resource_names
# package.get_resource('constituents-financials_csv').read()

['validation_report',
 'constituents_csv',
 'constituents-financials_csv',
 'constituents_json',
 'constituents-financials_json',
 's-and-p-500-companies-financials_zip',
 'constituents',
 'constituents-financials']

In [27]:
engine = create_engine('sqlite:///periodic-table-datapackage_company-wk.db')
package.save(storage='sql', engine=engine)

Storage <Engine(sqlite:///periodic-table-datapackage_company-wk.db)/None>

# 1970~2018年の株価データの確認

In [57]:
import pandas as pd
df = pd.read_csv('./resource_files/historical_stock_prices.csv.zip', sep=',', header=0)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20973889 entries, 0 to 20973888
Data columns (total 8 columns):
ticker       object
open         float64
close        float64
adj_close    float64
low          float64
high         float64
volume       int64
date         object
dtypes: float64(5), int64(1), object(2)
memory usage: 1.3+ GB


In [60]:
df.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [61]:
df.tail()

,ticker,open,close,adj_close,low,high,volume,date
20973884,NZF,14.60,14.59,14.59,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.58,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.59,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.57,14.57,14.64,135600,2018-08-23
20973888,NZF,14.60,14.69,14.69,14.59,14.69,180900,2018-08-24


In [64]:
df.agg({"date":[min,max]})

,date
min,1970-01-02
max,2018-08-24


In [63]:
df.query("ticker=='MMM'").agg({"date":[min,max]})

,date
min,1970-01-02
max,2018-08-24


# Macrotrendsからデータを抽出

## クローラーの準備

### 必要なパッケージのインストール

In [90]:
# !pip install selenium

In [16]:
import os
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# headless optionの設定
options = Options()
options.add_argument("--headless");
# driverの定義
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


## Macrotrendsへのクローラーの設定

In [4]:
url = "https://www.macrotrends.net/stocks/charts/"

In [5]:
# MacrotrendsのTopページへアクセス
driver.get(os.path.join(url, "MMM"))
target_url = driver.current_url

# 以降、Topページからの検索はjQueryでうまく実行できないため、銘柄ごとのURLを取得して、必要な情報のパスを追加してアクセスする

## データ取得処理のサンプル

In [298]:
## revenue
driver.get(os.path.join(target_url, "revenue"))
elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')

# サンプルの取得例
elements[0].text.split(" $")

['2020-09-30', '8,350']

In [307]:
list(map(lambda f: f.text, elements[0].find_elements_by_css_selector('td')))

['2020-09-30', '$8,350']

In [240]:
## revenue (rule 2)
driver.get(os.path.join(target_url, "revenue"))
elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')

# サンプルの取得例
# elements[0].text.split(" $")

# サンプルdfの生成例
df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
for i in range(0, len(elements)):
    indicator = "cost-goods-sold"
    date_val, val = elements[i].text.split(" $")
    df = df.append({'SYMBOL': "MMM", 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": "$"}, ignore_index=True)
df.tail()

,SYMBOL,INDICATOR,DATE,VALUE,UNIT
58,MMM,cost-goods-sold,2006-03-31,"5,595",$
59,MMM,cost-goods-sold,2005-12-31,"5,325",$
60,MMM,cost-goods-sold,2005-09-30,"5,382",$
61,MMM,cost-goods-sold,2005-06-30,"5,294",$
62,MMM,cost-goods-sold,2005-03-31,"5,166",$


In [242]:
## gross-margin (rule 1)
driver.get(os.path.join(target_url, "gross-margin"))
elements = driver.find_elements_by_css_selector('#style-1 > .table:nth-child(1) > tbody > tr')

# サンプルの取得例
# elements[0].text.split(" ")

# # サンプルdfの生成例
df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
for i in range(0, len(elements)):
    indicator = "cost-goods-sold"
    date_val, _, _, val = elements[i].text.split(" ")
    val = val.split("%")[0]
    df = df.append({'SYMBOL': "MMM", 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": "%"}, ignore_index=True)
df.tail()

,SYMBOL,INDICATOR,DATE,VALUE,UNIT
55,MMM,cost-goods-sold,2006-12-31,48.90,%
56,MMM,cost-goods-sold,2006-09-30,50.27,%
57,MMM,cost-goods-sold,2006-06-30,50.80,%
58,MMM,cost-goods-sold,2006-03-31,51.01,%
59,MMM,cost-goods-sold,2005-12-31,50.83,%


## まとめて取得するための設定

In [8]:
macrotrends_scrapying_rules = [
    {
        "CATEGORY": "Revenue&Profit",
        "INDICATORS": [
            {"INDICATOR": "revenue", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "gross-profit", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "operating-income", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "ebitda", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "net-income", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "eps-earnings-per-share-diluted", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "shares-outstanding", "RULE": 2, "UNIT": ""}]
    },
    {
        "CATEGORY": "Assets&Liabilityies",
        "INDICATORS": [
            {"INDICATOR": "total-assets", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "cash-on-hand", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "long-term-debt", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "total-liabilities", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "total-share-holder-equity", "RULE": 2, "UNIT": "$"}]
    },
    {
        "CATEGORY": "Margins",
        "INDICATORS": [
            {"INDICATOR": "profit-margins", "RULE": 2, "UNIT": "%"}
            , {"INDICATOR": "gross-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "operating-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "ebitda-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "pre-tax-profit-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "net-profit-margin", "RULE": 1, "UNIT": "%"}]
    },
    {
        "CATEGORY": "Price_Ratios",
        "INDICATORS": [
            {"INDICATOR": "pe-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-sales", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-book", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-fcf", "RULE": 1, "UNIT": ""}]
    },
    {
        "CATEGORY": "Other_Ratios",
        "INDICATORS": [
            {"INDICATOR": "current-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "quick-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "debt-equity-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "roe", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "roa", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "roi", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "return-on-tangible-equity", "RULE": 1, "UNIT": "%"}]
    }
]

# S&P銘柄の情報をmacrotrendsから取得する関数を定義

In [10]:
## 必要なパッケージのインストール
import numpy as np
import time

In [11]:
# 銘柄の情報を引数のDataFrameへ登録
## dfはのcolumnsは、["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"]を想定。（それ以外の場合はエラー）
def addMacrotrendsInfoBySymbol(symbol, driver, macrotrends_rules):
    df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
    base_url = "https://www.macrotrends.net/stocks/charts/"
    driver.get(os.path.join(base_url, symbol))
    target_url = driver.current_url
    
    start = time.time()
    rap_start = start
    print("# [START] symbol: {0}".format(symbol))
    for i in range(0, len(macrotrends_rules)):
        for j in range(0, len(macrotrends_rules[i]["INDICATORS"])):
            indicator, rule, unit = macrotrends_rules[i]["INDICATORS"][j].values()
            if rule == 1:
                driver.get(os.path.join(target_url, indicator))
                elements = driver.find_elements_by_css_selector('#style-1 > .table:nth-child(1) > tbody > tr')
                for k in range(0, len(elements)):
                    date_val, _, _, val = list(map(lambda f: f.text, elements[k].find_elements_by_css_selector('td')))
                    val = val.replace(unit, "")
                    df = df.append({'SYMBOL': symbol, 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": unit}, ignore_index=True)
            else:
                if rule == 2:
                    driver.get(os.path.join(target_url, indicator))
                    elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')
                    for k in range(0, len(elements)):
                        date_val, val = list(map(lambda f: f.text, elements[k].find_elements_by_css_selector('td')))
                        val = val.replace(unit, "")
                        df = df.append({'SYMBOL': symbol, 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": unit}, ignore_index=True)
                else:
                    raise Exception("macrotrends_rules is incorrect. rule: {0}, {1}".format(indicator, rule))
            
#             elapsed_time = time.time() - rap_start
#             print ("## [RAP] indicator: {0} / elapsed_time: {1:.2f}".format(indicator, elapsed_time) + "[sec]")
#             rap_start = time.time()
    print("# [END] symbol: {0} / total_time: {1:.2f}".format(symbol, time.time() - start) + "[sec]")
    return df

# S&Pのsymbolを取得

In [17]:
df_s_and_p_stock_list = pd.read_csv("./output/s_and_p_stock_list.csv", sep=",", header=0)

In [18]:
s_and_p_stock_list = df_s_and_p_stock_list["Symbol"].unique()

In [19]:
len(s_and_p_stock_list)

505

# 1970~2018年のストックデータに含まれる銘柄の確認

In [375]:
df_historical_stocks = pd.read_csv("./resource_files/historical_stock_prices.csv.zip", sep=",", header=0)

In [376]:
df_historical_stocks.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [377]:
df_historical_stocks[["ticker"]] \
    .drop_duplicates() \
    .query("ticker in @s_and_p_stock_list") \
    .shape

(484, 1)

# カテゴリごとにデータを取得

In [20]:
## 事前定義のオブジェクトを準備
options = Options()
options.add_argument("--headless");
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)
# df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [1]:
# index = 0
# rules = [macrotrends_scrapying_rules[index]]
# category = rules[0]["CATEGORY"]

# ## 実行
# for i in range(0, len(s_and_p_stock_list)):
#     df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
# df.to_csv("./output/macrotrends_{0}.csv".format(category))

In [2]:
# index = 1
# rules = [macrotrends_scrapying_rules[index]]
# category = rules[0]["CATEGORY"]

# ## 実行
# for i in range(0, len(s_and_p_stock_list)):
#     df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
# df.to_csv("./output/macrotrends_{0}.csv".format(category))

In [3]:
# index = 2
# rules = [macrotrends_scrapying_rules[index]]
# category = rules[0]["CATEGORY"]

# ## 実行
# for i in range(0, len(s_and_p_stock_list)):
#     df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
# df.to_csv("./output/macrotrends_{0}.csv".format(category))

In [4]:
# index = 3
# rules = [macrotrends_scrapying_rules[index]]
# category = rules[0]["CATEGORY"]

# ## 実行
# for i in range(0, len(s_and_p_stock_list)):
#     df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
# df.to_csv("./output/macrotrends_{0}.csv".format(category))

In [5]:
# index = 4
# rules = [macrotrends_scrapying_rules[index]]
# category = rules[0]["CATEGORY"]

# ## 実行
# for i in range(443, len(s_and_p_stock_list)):
#     df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
# df.to_csv("./output/macrotrends_{0}.csv".format(category))